<a href="https://colab.research.google.com/github/yiwasaki89/my-timer-app/blob/main/LLM_sosiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#base_folder = "/content/drive/MyDrive/製造業対抗データインサイトチャレンジ/data/"
base_folder = "/content/drive/MyDrive/Colab Notebooks/製造業対抗データインサイトチャレンジ/data/"
train = pd.read_csv(base_folder + "train.csv")
test = pd.read_csv(base_folder + "test.csv")

In [ ]:
print('train', train.shape,'test', test.shape)

train (742, 43) test (800, 42)


In [ ]:
# =============================
# 「組織図」→ Org_DX_structure_flag_llm (0/1)
# =============================

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 組織図の列名（必要なら変更）
org_col = "組織図"

# 生成パラメータ（テンプレ側に既定があればそちらが優先されます）
max_new_tokens = globals().get("max_new_tokens", 64)
temperature    = globals().get("temperature", 0.3)
top_p          = globals().get("top_p", 0.9)
max_retries    = globals().get("max_retries", 2)

# モデル設定（Qwenなど） - Assuming these are needed for the functions below
model_name = "Qwen/Qwen3-1.7B"   # 必要なら他モデルに変更
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device)
model_max_length = 4096
context_margin = 256


def _trim_to_context(text: str, max_len: int) -> str:
    if text is None:
        return ""
    text = str(text)
    return text[-max_len:] if len(text) > max_len else text

def make_prompt_org(org_text: str) -> str:
    """
    「組織図」テキストを読み、#組織 の要件に一致するか判定して JSON を返す。
    #組織:
      DXを推進するための組織がある。
    """
    rubric = """
あなたは日本語に長けたDX推進を行うアナリストです。
与えられた「組織図」や組織記述を読み、次の要件に合致するか判定してください。

#要件:
- DXを推進するための専門組織（例：DX推進室、CDO直轄のDX本部、デジタル戦略部、データ戦略室など）がある

出力は次のJSONのみで、余計な文章を含めないこと：
{"Org_DX_structure_flag": <1|0>}

ルール:
- 明確に該当の組織/役員の存在・配置が読み取れる場合は 1
- 明確な記載がない、曖昧、または否定的な場合は 0
必ず上記のJSONだけを出力してください。
"""
    sample = f'【組織記述】\n{org_text}\n'
    return rubric + "\n" + sample

@torch.inference_mode()
def _generate_text(prompt: str,
                   max_new_tokens=max_new_tokens,
                   temperature=temperature,
                   top_p=top_p,
                   num_retries=max_retries) -> str:
    """Qwen等で生成（簡易リトライ付）"""
    input_ids = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=min(int(model_max_length) - 64, 4096)
    ).to(device)

    for _ in range(max(1, num_retries)):
        outputs = model.generate(
            **input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=(temperature > 0),
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if text.startswith(prompt):
            text = text[len(prompt):]
        text = text.strip()
        if text:
            return text
    return ""

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# =============================
# 「組織図」判定：生成→保存 & 読み出しユーティリティ
# =============================

import os
import pandas as pd

def process_and_save(input_csv_path: str,
                     output_csv_path: str,
                     org_col_name: str = "組織図",
                     num_rows_to_process: int | None = None,
                     index: bool = False) -> pd.DataFrame:
    """入力CSVを読み込み、LLMで組織図を評価→出力"""
    if not os.path.exists(input_csv_path):
        raise FileNotFoundError(f"input_csv_path not found: {input_csv_path}")

    df = pd.read_csv(input_csv_path)
    df = infer_org_flag_for_df(df, org_col_name=org_col_name, num_rows_to_process=num_rows_to_process)

    out_dir = os.path.dirname(output_csv_path)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)

    df.to_csv(output_csv_path, index=index)
    return df


def process_batch(files: dict,
                  org_col_name: str = "組織図",
                  num_rows_to_process: int | None = None,
                  index: bool = False) -> dict:
    """複数ファイルを一括処理（train/testなど）"""
    results = {}
    for k, (inp, out) in files.items():
        results[k] = process_and_save(
            input_csv_path=inp,
            output_csv_path=out,
            org_col_name=org_col_name,
            num_rows_to_process=num_rows_to_process,
            index=index
        )
    return results


def read_processed(output_csv_path: str) -> pd.DataFrame:
    """保存済みCSVを読み込んで簡易確認"""
    if not os.path.exists(output_csv_path):
        raise FileNotFoundError(f"processed csv not found: {output_csv_path}")

    df = pd.read_csv(output_csv_path)
    col = "Org_DX_structure_flag_llm"
    if col in df.columns:
        vc = df[col].value_counts(dropna=False).sort_index()
        print(f"[{col}] value_counts:\n{vc}\n")
    else:
        print(f"[WARN] '{col}' が見つかりませんでした。")
    return df


In [ ]:
# ===========================================================
# Google Colab対応：組織図→Org_DX_structure_flag_llm 判定一式
# ===========================================================

import os
import json
import re
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

# -----------------------------
# モデル設定（Qwenなど）
# -----------------------------
model_name = "Qwen/Qwen3-1.7B"   # 必要なら他モデルに変更
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device)
model_max_length = 4096
context_margin = 256

max_new_tokens = 64
temperature = 0.3
top_p = 0.9
max_retries = 2

# -----------------------------
# 組織図判定ロジック
# -----------------------------
org_col = "組織図"

def _trim_to_context(text: str, max_len: int) -> str:
    if text is None:
        return ""
    text = str(text)
    return text[-max_len:] if len(text) > max_len else text

def make_prompt_org(org_text: str) -> str:
    rubric = """
あなたは日本語のテキスト評価アシスタントです。
与えられた「組織図」や組織記述を読み、次の要件に合致するか判定してください。

#要件:
- DXを推進するために組織上位置付けられた専門組織（例：CDO直轄のDX本部、デジタル戦略部、データ戦略室など）がある

出力は次のJSONのみで、余計な文章を含めないこと：
{"Org_DX_structure_flag": <1|0>}

ルール:
- 明確に該当の組織/役員の存在・配置が読み取れる場合は 1
- 明確な記載がない、曖昧、または否定的な場合は 0
必ず上記のJSONだけを出力してください。
"""
    return rubric + f"\n【組織記述】\n{org_text}\n"

@torch.inference_mode()
def _generate_text(prompt: str) -> str:
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=model_max_length - 64).to(device)
    for _ in range(max_retries):
        outputs = model.generate(
            **input_ids,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if text.startswith(prompt):
            text = text[len(prompt):]
        text = text.strip()
        if text:
            return text
    return ""

def _extract_json_dict(text: str):
    m = re.search(r"\{[\s\S]*?\}", text)
    if not m:
        return None
    try:
        return json.loads(m.group(0))
    except Exception:
        return None

def _heuristic_org_flag(text: str) -> int:
    """フォールバック：キーワードヒューリスティック"""
    if not text:
        return 0
    txt = str(text)
    keywords_any = [
        "DX推進室","DX本部","DX推進本部","DX推進部","デジタル戦略部","データ戦略室",
        "デジタル推進部","デジタル統括","デジタルイノベーション","CDO",
        "CDXO","デジタル担当役員","Chief Digital","Chief Data","DX担当執行役員"
    ]
    return 1 if any(k in txt for k in keywords_any) else 0

def infer_org_flag_for_df(df, org_col_name: str = org_col, num_rows_to_process=None):
    """DataFrameの組織図列にフラグ付与"""
    tqdm.pandas()
    if org_col_name not in df.columns:
        df["Org_DX_structure_flag_llm"] = np.nan
        df["_qwen_raw_output_org"] = ""
        return df

    df_proc = df.iloc[:num_rows_to_process].copy() if num_rows_to_process else df.copy()

    def _apply_row(text):
        prompt = make_prompt_org(_trim_to_context(text, model_max_length - context_margin))
        out = _generate_text(prompt)
        parsed = _extract_json_dict(out) or {}
        org_flag = parsed.get("Org_DX_structure_flag", None)

        if org_flag in [0, 1, "0", "1"]:
            org_flag = int(org_flag)
        else:
            org_flag = _heuristic_org_flag(text)
        return pd.Series({
            "_qwen_raw_output_org": out,
            "Org_DX_structure_flag_llm": org_flag
        })

    res = df_proc[org_col_name].progress_apply(_apply_row)
    for c in res.columns:
        df_proc[c] = res[c]
    df.loc[df_proc.index, ["Org_DX_structure_flag_llm", "_qwen_raw_output_org"]] = \
        df_proc[["Org_DX_structure_flag_llm", "_qwen_raw_output_org"]]
    return df

# -----------------------------
# 保存・読込ユーティリティ
# -----------------------------
def process_and_save(input_csv_path: str,
                     output_csv_path: str,
                     org_col_name: str = "組織図",
                     num_rows_to_process=None,
                     index=False) -> pd.DataFrame:
    """組織図判定→CSV保存"""
    if not os.path.exists(input_csv_path):
        raise FileNotFoundError(f"input_csv_path not found: {input_csv_path}")

    df = pd.read_csv(input_csv_path)
    df = infer_org_flag_for_df(df, org_col_name=org_col_name, num_rows_to_process=num_rows_to_process)

    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
    df.to_csv(output_csv_path, index=index)
    print(f"✅ Saved: {output_csv_path}")
    return df

def read_processed(output_csv_path: str) -> pd.DataFrame:
    """保存済みCSVを確認"""
    if not os.path.exists(output_csv_path):
        raise FileNotFoundError(f"processed csv not found: {output_csv_path}")

    df = pd.read_csv(output_csv_path)
    col = "Org_DX_structure_flag_llm"
    if col in df.columns:
        vc = df[col].value_counts(dropna=False).sort_index()
        print(f"[{col}] 分布:\n{vc}\n")
    return df


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
base = "/content/drive/MyDrive/Colab Notebooks/製造業対抗データインサイトチャレンジ/data/"
input_train = base + "train.csv"
output_train = base + "train_with_org_flag.csv"

df_train = process_and_save(
    input_csv_path=input_train,
    output_csv_path=output_train,
    org_col_name="組織図",
    num_rows_to_process=None,
    index=False
)

# 確認
df_check = read_processed(output_train)
df_check.head()

100%|██████████| 742/742 [38:43<00:00,  3.13s/it]


✅ Saved: /content/drive/MyDrive/Colab Notebooks/製造業対抗データインサイトチャレンジ/data/train_with_org_flag.csv
[Org_DX_structure_flag_llm] 分布:
Org_DX_structure_flag_llm
0.0    392
1.0    350
Name: count, dtype: int64



,企業ID,企業名,従業員数,業界,上場種別,特徴,企業概要,組織図,事業所数,工場数,...,アンケート６,アンケート７,アンケート８,アンケート９,アンケート１０,アンケート１１,今後のDX展望,購入フラグ,Org_DX_structure_flag_llm,_qwen_raw_output_org
0,0,株式会社クロマファクトリー,4539,化学,PR,BtoB,【会社概要】\n\n当社は、化学業界に属し、先進の技術と豊富な実績を背景に、国内外のお客様に...,【経営層】\n├─ 代表取締役社長\n├─ 取締役副社長\n└─ 取締役\n\n【管理本部】...,2.0,10.0,...,2,NaN,3,4,1,1,弊社は、既存事業基盤を支える先端技術と品質管理体制をさらに磨くため、DX投資を持続可能かつ実...,0,0.0,└─ 技術サポート課\n\n【デジタル戦略部】\n└─ デジタル戦略課\n\n【情報技術部】...
1,1,星浪ホールディングス株式会社,792,運輸・物流,PR,BtoB,当社は運輸・物流業界に属し、792名の従業員と共に、バスや鉄道輸送、物流・海運、不動産・百貨...,経営層\n├─ 経営管理部\n│ ├─ 経営企画室\n│ └─ 財務・人事部\n├─...,2.0,NaN,...,1,5.0,4,1,2,1,当グループは運輸・物流、不動産・小売、ホテル・文化事業など幅広い事業を抱えるものの、これまで...,0,0.0,※この組織構成は、経営層の下に設置された各事業本部の構成を示しています。\n\n【組織図】\...
2,2,株式会社クネットリンク,4346,建設・工事,PR,BtoB,2010年に3社の経営統合により持株会社として設立され、業界3位を誇る電気通信工事会社です。...,経営層\n├─ 経営企画室\n├─ 管理本部\n│ ├─ 人事部\n│ ├─ 財務部\n│ ...,24.0,NaN,...,1,4.0,4,1,1,2,弊社はこれまで、社内DX導入において業界屈指の前のめり姿勢を貫き、AI・IoT・クラウドなど...,0,0.0,```\nこの組織図では、技術本部が「設計・開発部」、「施工管理部」、「新技術開発部」を含む...
3,3,株式会社プリントスフィア,2011,機械,PR,BtoB,"【企業概要】\n当社は、従業員数2,011名を有し、機械業界に属するグローバル企業として、製...",【企業組織図】\n\n■ 経営層\n ├─ 経営企画部\n ├─ 自動認識ソリューション事業...,1.0,NaN,...,2,NaN,1,2,3,4,当社はこれまで、クラウド、AI、IoT などを活用した製造プロセス・サプライチェーンの効率化...,0,0.0,■ 事業部\n ├─ 事業部A\n │ ├─ メカトロ製品部門\n │ │ ├─ 製造部\n...
4,4,株式会社悠薫薬品,1488,医療・福祉,PR,BtoB,従業員数：1488人\n業界：医療・福祉業界\n\n事業内容：\n当社は、先発医薬品としての...,────────────────────────────\n【経営層】\n ├─ 代表取締役...,58.0,2.0,...,1,5.0,2,3,5,4,弊社はこれまで、クラウド、AI、IoT などを計画的に導入し、業務効率化や製品開発の高度化に...,0,0.0,【組織図】\n ├─ 代表取�ート社長／経営陣\n └─ 経営企画室\n ├─ 人事部...


In [ ]:
import pandas as pd
import os

# Load the processed data
base_folder = "/content/drive/MyDrive/Colab Notebooks/製造業対抗データインサイトチャレンジ/data/"
processed_train_path = os.path.join(base_folder, "train_with_org_flag.csv")
df_processed_train = pd.read_csv(processed_train_path)

# Select the required columns
df_output = df_processed_train[['企業ID', 'Org_DX_structure_flag_llm']]

# Define the output path for the new CSV
output_csv_path = os.path.join(base_folder, "train_org_flag_output.csv")

# Save the selected columns to a new CSV file
df_output.to_csv(output_csv_path, index=False)

print(f"Successfully saved '企業ID' and 'Org_DX_structure_flag_llm' to {output_csv_path}")

Successfully saved '企業ID' and 'Org_DX_structure_flag_llm' to /content/drive/MyDrive/Colab Notebooks/製造業対抗データインサイトチャレンジ/data/train_org_flag_output.csv


In [ ]:
# クロス集計表の作成
cross_tab = pd.crosstab(df_processed_train['購入フラグ'], df_processed_train['Org_DX_structure_flag_llm'])

# 結果の表示
print("「購入フラグ」と「Org_DX_structure_flag_llm」のクロス集計表:")
display(cross_tab)

「購入フラグ」と「Org_DX_structure_flag_llm」のクロス集計表:


Org_DX_structure_flag_llm,0.0,1.0
購入フラグ,,
0,321,242
1,71,108


In [ ]:
base = "/content/drive/MyDrive/Colab Notebooks/製造業対抗データインサイトチャレンジ/data/"
input_test = base + "test.csv"
output_test = base + "test_with_org_flag.csv"

df_test = process_and_save(
    input_csv_path=input_test,
    output_csv_path=output_test,
    org_col_name="組織図",
    num_rows_to_process=None,
    index=False
)

# 確認
df_check_test = read_processed(output_test)
df_check_test.head()

100%|██████████| 800/800 [41:43<00:00,  3.13s/it]


✅ Saved: /content/drive/MyDrive/Colab Notebooks/製造業対抗データインサイトチャレンジ/data/test_with_org_flag.csv
[Org_DX_structure_flag_llm] 分布:
Org_DX_structure_flag_llm
0.0    392
1.0    408
Name: count, dtype: int64



,企業ID,企業名,従業員数,業界,上場種別,特徴,企業概要,組織図,事業所数,工場数,...,アンケート５,アンケート６,アンケート７,アンケート８,アンケート９,アンケート１０,アンケート１１,今後のDX展望,Org_DX_structure_flag_llm,_qwen_raw_output_org
0,742,株式会社ヒトリンクステーション,847,人材,PR,BtoB,当社は、人材業界に属する持株会社として、従業員数847名を擁し、グループ全体の資源活用と戦略...,営業層\n├─ 経営企画部\n│ ├─ 戦略企画室\n│ └─ DX推進室\n├─ ...,209.0,NaN,...,5,1,5.0,2,2,2,4,弊社グループは、既存事業とのシナジーを意識しながらDXに取り組んでまいりましたが、これまでは...,1.0,```\n\nこの組織図では、DX推進室が営業層の下にある。DX推進室は営業層の下にある。こ...
1,743,トワリズム株式会社,505,アパレル・美容,PR,"BtoB, BtoC",当社は、従業員数約505名を擁する、アパレル・美容業界をはじめとした多角的事業を展開する企業...,#組織図\n経営層\n├─ 代表取締役社長\n├─ 副社長\n└─ 経営企画・管理本部\n ...,10.0,1.0,...,2,2,NaN,3,1,5,4,当社はこれまで、部門横断での基幹システム刷新やAI・クラウド活用など、社内業務のデジタル化を...,0.0,・ デジタル戦略部\n\n└─────────────────────────────┘\n...
2,744,音心楽舞株式会社,1415,エンタメ,PR,"BtoB, BtoC",当社はエンタメ業界において独自の企画力とクリエイティビティで、音楽分野でも首位級の業績を誇る...,■ 経営層 \n └─ 代表取締役社長\n\n │ \n ├─ 経営企画部 \...,1.0,NaN,...,4,1,1.0,3,1,4,3,弊社ではDX推進を掲げつつも、これまでは社内導入を限定的な範囲にとどめ、既存プラットフォーム...,1.0,■ 事業部 \n └─ デジタルコンテンツ事業部 \n └─ デジタルコンテンツ事業...
3,745,株式会社グローバル・ケアリンク,4008,人材,PR,BtoB,従業員4008名を擁する当社は、人材業界を基盤に、損保・自動車関連の顧客対応、海外・クレジッ...,【経営層】\n├─ 経営企画室\n│ └─ DX推進室\n├─ 財務管理部\n└─ 事業...,10.0,NaN,...,3,1,3.0,1,2,3,1,弊社はこれまで、経営層がDXの重要性を共有しつつも、全社一丸となった明確なロードマップまでは...,1.0,【経営層】\n└─ 経営企画室\n └─ DX推進室\n【経営層】\n └─ 経...
4,746,株式会社アクアリーバ,2647,食品,PR,BtoB,当グループは、食品業界を基盤としながら水産および医薬・機能性原料事業を中核として、多角的な事...,経営層\n├─ 経営企画室\n├─ 財務・管理部\n└─ 事業本部\n ├─ 食品事業...,10.0,7.0,...,3,2,NaN,3,2,2,3,当社はこれまで、クラウド、AI、IoT などの先端技術を活用しながら、部門ごとの課題に応じて...,0.0,└─ 経営戦略部\n ├─ 経営戦略課\n └─ 経営分析課\n└─ 経営企画室...


In [ ]:
import pandas as pd
import os

# Load the processed data
base_folder = "/content/drive/MyDrive/Colab Notebooks/製造業対抗データインサイトチャレンジ/data/"
processed_test_path = os.path.join(base_folder, "test_with_org_flag.csv")
df_processed_test = pd.read_csv(processed_test_path)

# Select the required columns
df_output_test = df_processed_test[['企業ID', 'Org_DX_structure_flag_llm']]

# Define the output path for the new CSV
output_csv_path_test = os.path.join(base_folder, "test_org_flag_output.csv")

# Save the selected columns to a new CSV file
df_output_test.to_csv(output_csv_path_test, index=False)

print(f"Successfully saved '企業ID' and 'Org_DX_structure_flag_llm' for the test data to {output_csv_path_test}")

Successfully saved '企業ID' and 'Org_DX_structure_flag_llm' for the test data to /content/drive/MyDrive/Colab Notebooks/製造業対抗データインサイトチャレンジ/data/test_org_flag_output.csv
